In [1]:
import sys
sys.path.insert(0,'C:\\Users\\Syahrir Ridha\\PycharmProjects\\NET_Solver\\')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from models import*
from hard_boundary import *

In [2]:
def radial_bipolar(r_out, r_inn, ecc):
    r = r_inn/r_out
    xi_out = np.arccosh( (r*(1+ecc**2) + (1-ecc**2)) / (2*r*ecc) )
    xi_inn = np.arccosh( (r*(1-ecc**2) + (1+ecc**2)) / (2*ecc) )
    c_1 = r_inn*np.sinh(xi_out)
    c_2 = r_out*np.sinh(xi_inn)
    np.testing.assert_allclose(c_1, c_2, rtol=1e-10, atol=0)
    return xi_out, xi_inn, c_1

In [3]:
r_out = 1.0
r_inn = 0.6
ecc = 0.001
xi_out, xi_inn, c = radial_bipolar(r_out,r_inn, ecc)
print(f'Outer Xi: {xi_out} Inner Xi: {xi_inn} Focal Point: {c}')

Outer Xi: 7.8885841413687325 Inner Xi: 7.377758767602808 Focal Point: 799.9995749999124


In [4]:
# define the mesh
nx = 8
ny = 16
xi_, eta_ = np.linspace(xi_inn, xi_out, nx), np.linspace(0, np.pi, ny)
xi, eta = np.meshgrid(xi_, eta_)

In [5]:
# defne neural network
hid_dim = 100
layers = 4
in_dim = 2
out_dim = 1
act = torch.nn.Tanh()
net = MLP(in_dim, out_dim, layers, hid_dim, act = act)

In [6]:
# prepare input data for neural network
xi, eta = torch.Tensor(xi).float().reshape(-1,1), torch.Tensor(eta).float().reshape(-1,1)
input_data = torch.cat([xi,eta], dim=1)

In [7]:
# neural network output
out = net(input_data)

In [8]:
# when xi = xi_inn
test_xi_inn = input_data[:,0] <= xi_inn
test_xi_inn_indi = torch.nonzero(test_xi_inn).unsqueeze(1)
out[test_xi_inn_indi] = 0.0

# when xi = xi_out
test_xi_out = input_data[:,0] >= xi_out
test_xi_out_indi = torch.nonzero(test_xi_out).unsqueeze(1)
out[test_xi_out_indi] = 0.0

# when eta = 0
test_eta_in = input_data[:,1] <= 0.0
test_eta_in_indi = torch.nonzero(test_eta_in).unsqueeze(1)
out[test_eta_in_indi] = out[test_eta_in_indi+ny]

# when eta = pi
test_eta_out = input_data[:,1] >= np.pi
test_eta_out_indi = torch.nonzero(test_eta_out).unsqueeze(1)
out[test_eta_out_indi] = out[test_eta_out_indi-ny]

In [9]:
torch.cat([test_xi_inn_indi.reshape(-1,1), test_xi_out_indi.reshape(-1,1), test_eta_in_indi.reshape(-1,1), test_eta_out_indi.reshape(-1,1)], dim=1)

RuntimeError: Sizes of tensors must match except in dimension 1. Got 16 and 8 in dimension 0 (The offending index is 2)

In [10]:
class Trial_Solution:
    def __init__(self, model, boundary):
        self.model = model
        self.model.apply(_init_weights)
        self.var = tuple(boundary.keys())
        self.bound = list(boundary.values())
        
    def __call__(self,input_data):
        
        # prediction from neural network
        output_data = self.model(input_data)
        
        # when input is equal at the boundary data
        xi_inn = input_data[:,0] <= self.bound[0]
        xi_out = input_data[:,0] >= self.bound[1]
        
        # find index of both the boundary 
        xi_inn_index = torch.nonzero(xi_inn).unsqueeze(1)
        xi_out_index = torch.nonzero(xi_out).unsqueeze(1)
        
        # filter the output
        output_data[xi_inn_index] = 0.0
        output_data[xi_out_index] = 0.0
        return output_data

In [11]:
Trial_Solution(net,{'xi_inn':xi_inn, 'xi_out':xi_out})(input_data).reshape(ny,nx)

tensor([[0.0000, 0.0705, 0.0707, 0.0709, 0.0710, 0.0712, 0.0713, 0.0000],
        [0.0000, 0.0720, 0.0721, 0.0723, 0.0724, 0.0726, 0.0727, 0.0000],
        [0.0000, 0.0734, 0.0735, 0.0736, 0.0738, 0.0739, 0.0740, 0.0000],
        [0.0000, 0.0748, 0.0749, 0.0750, 0.0751, 0.0752, 0.0753, 0.0000],
        [0.0000, 0.0762, 0.0763, 0.0764, 0.0765, 0.0765, 0.0766, 0.0000],
        [0.0000, 0.0778, 0.0779, 0.0779, 0.0779, 0.0780, 0.0780, 0.0000],
        [0.0000, 0.0795, 0.0795, 0.0795, 0.0796, 0.0796, 0.0796, 0.0000],
        [0.0000, 0.0814, 0.0814, 0.0814, 0.0813, 0.0813, 0.0813, 0.0000],
        [0.0000, 0.0834, 0.0834, 0.0833, 0.0833, 0.0832, 0.0832, 0.0000],
        [0.0000, 0.0856, 0.0855, 0.0854, 0.0853, 0.0853, 0.0852, 0.0000],
        [0.0000, 0.0878, 0.0877, 0.0876, 0.0875, 0.0874, 0.0873, 0.0000],
        [0.0000, 0.0901, 0.0900, 0.0898, 0.0897, 0.0896, 0.0895, 0.0000],
        [0.0000, 0.0924, 0.0923, 0.0921, 0.0920, 0.0919, 0.0917, 0.0000],
        [0.0000, 0.0948, 0.0946, 0.094

In [ ]:
tuple(dic.keys())

In [ ]:
diri =  list(dic.values())
diri

In [ ]:
for i in diri:
    print(i)